In [55]:
model_path = '/content/qat_model.tflite'

In [ ]:
pip install tensorflow flatbuffers

In [ ]:
!sudo mv /content/flatc /usr/local/bin/
!sudo chmod +x /usr/local/bin/flatc
!flatc --python schema_v3c.fbs

In [ ]:
!flatc --version

flatc version 24.3.25


#start

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
import tflite.BuiltinOperator as BuiltinOperator
import tflite.Operator as Operator
import tflite.TensorType as TensorType
import tflite.Conv2DOptions as Conv2DOptions

In [ ]:
import tflite.Model as Model
import tflite.TensorType as TensorType
import tflite.Conv2DOptions as Conv2DOptions
import tflite.BuiltinOperator as BuiltinOperator
import tflite.PadOptions as PadOptions
import tflite.TransposeOptions as TransposeOptions
import tflite.PadV2Options as PadV2Options
import tflite.Pool2DOptions as Pool2DOptions
import tflite.ConcatenationOptions as ConcatenationOptions
import tflite.SplitOptions as SplitOptions
import tflite.ResizeNearestNeighborOptions as ResizeNearestNeighborOptions

In [ ]:
!pip install tflite-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow flatbuffers

In [ ]:
import tflite_runtime.interpreter as tflite

In [ ]:
import tensorflow as tf
import numpy as np
import flatbuffers
# import tflife.Model as Model

# Import the generated schema modules
# import schema_generated_python.tflite as tflite

# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path="/content/qat_model.tflite")
interpreter.allocate_tensors()

# # Get tensor details
tensor_details = interpreter.get_tensor_details()

# Get the model content
with open("/content/qat_model.tflite", "rb") as f:
    model_content = f.read()

# Parse the model using flatbuffers
model = Model.Model.GetRootAsModel(model_content, 0)
subgraph = model.Subgraphs(0)

# Function to extract information about convolutional layers and map tensor names
def extract_conv_layers_details(subgraph):
    conv_layers_info = []

    for i in range(subgraph.OperatorsLength()):
        op = subgraph.Operators(i)
        op_code = model.OperatorCodes(op.OpcodeIndex()).BuiltinCode()

        # Check if the operator is a convolutional layer
        if op_code == BuiltinOperator.BuiltinOperator.CONV_2D:
            layer_info = {}
            layer_info['name'] = f"CONV_2D_{i}"
            layer_info['index'] = i

            # Extract input and output tensor indices
            input_tensor_idx = op.Inputs(0)
            weight_tensor_idx = op.Inputs(1)
            output_tensor_idx = op.Outputs(0)

            # Extract weight details
            weight_tensor = subgraph.Tensors(weight_tensor_idx)
            weights_shape = weight_tensor.ShapeAsNumpy()
            layer_info['weights_shape'] = weights_shape
            layer_info['weights_dtype'] =0

            # Extract padding and stride details from options
            options = op.BuiltinOptions()
            conv_options = Conv2DOptions.Conv2DOptions()
            conv_options.Init(options.Bytes, options.Pos)
            layer_info['padding'] = conv_options.Padding()
            layer_info['stride_h'] = conv_options.StrideH()
            layer_info['stride_w'] = conv_options.StrideW()

            # Map tensor names
            layer_info['input_tensor_name'] = tensor_details[input_tensor_idx]['name']
            layer_info['weight_tensor_name'] = tensor_details[weight_tensor_idx]['name']
            layer_info['bias_tensor_name'] = tensor_details[weight_tensor_idx+1]['name']
            layer_info['output_tensor_name'] = tensor_details[output_tensor_idx]['name']

            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.QUANTIZE:
            layer_info = {}
            layer_info['name'] = f"QUANTIZE_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name'] = tensor_details[op.Inputs(0)]['name']
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.DEQUANTIZE:
            layer_info = {}
            layer_info['name'] = f"DEQUANTIZE_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name'] = tensor_details[op.Inputs(0)]['name']
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.ADD:
            layer_info = {}
            layer_info['name'] = f"ADD_{i}"
            layer_info['index'] = i

            layer_info['input_tensor_name'] = [tensor_details[op.Inputs(j)]['name'] for j in range(op.InputsLength())]
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.RESHAPE:
            layer_info = {}
            layer_info['name'] = f"RESHAPE_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name'] = tensor_details[op.Inputs(0)]['name']
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            options = op.BuiltinOptions()
            reshape_options = ReshapeOptions.ReshapeOptions()
            reshape_options.Init(options.Bytes, options.Pos)
            layer_info['shape'] = reshape_options.Shape()

            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.SPLIT:
            layer_info = {}
            layer_info['name'] = f"SPLIT_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name'] = tensor_details[op.Inputs(0)]['name']
            layer_info['output_tensor_name'] = [tensor_details[op.Outputs(j)]['name'] for j in range(op.OutputsLength())]
            conv_layers_info.append(layer_info)
        if op_code == BuiltinOperator.BuiltinOperator.RESIZE_NEAREST_NEIGHBOR:
            layer_info = {}
            layer_info['name'] = f"UPSAMPLE_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name']  = [tensor_details[op.Inputs(j)]['name'] for j in range(op.InputsLength())]
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            # options = op.BuiltinOptions()
            # resize_options = ResizeNearestNeighborOptions.ResizeNearestNeighborOptions()
            # resize_options.Init(options.Bytes, options.Pos)
            # layer_info['scale'] = resize_options.Scale()
            # layer_info['align_corners'] = resize_options.AlignCorners()
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.TRANSPOSE:
            layer_info = {}
            layer_info['name'] = f"TRANPOSE_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name']  = [tensor_details[op.Inputs(j)]['name'] for j in range(op.InputsLength())]
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            # options = op.BuiltinOptions()
            # transpose_options = TransposeOptions.TransposeOptions()
            # transpose_options.Init(options.Bytes, options.Pos)
            # layer_info['perm'] = transpose_options.Perm()
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.CONCATENATION:
            layer_info = {}
            layer_info['name'] = f"CONCAT_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name']  = [tensor_details[op.Inputs(j)]['name'] for j in range(op.InputsLength())]

            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.MAX_POOL_2D:
            layer_info = {}
            layer_info['name'] = f"MAXPOOL_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name'] = tensor_details[op.Inputs(0)]['name']
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            options = op.BuiltinOptions()
            pool_options = Pool2DOptions.Pool2DOptions()
            pool_options.Init(options.Bytes, options.Pos)
            layer_info['padding'] = pool_options.Padding()
            layer_info['stride_h'] = pool_options.StrideH()
            layer_info['stride_w'] = pool_options.StrideW()
            layer_info['kernel_h'] = pool_options.FilterHeight()
            layer_info['kernel_w'] = pool_options.FilterWidth()

            conv_layers_info.append(layer_info)


        if op_code == BuiltinOperator.BuiltinOperator.PAD:
            layer_info = {}
            layer_info['name'] = f"PAD_{i}"
            layer_info['index'] = i

            layer_info['input_tensor_name']  = [tensor_details[op.Inputs(j)]['name'] for j in range(op.InputsLength())]

            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            conv_layers_info.append(layer_info)

        if op_code == BuiltinOperator.BuiltinOperator.PADV2:
            layer_info = {}
            layer_info['name'] = f"PADV2_{i}"
            layer_info['index'] = i
            layer_info['input_tensor_name']  = [tensor_details[op.Inputs(j)]['name'] for j in range(op.InputsLength())]
            layer_info['output_tensor_name'] = tensor_details[op.Outputs(0)]['name']
            conv_layers_info.append(layer_info)


    return conv_layers_info

# Extract and print convolutional layer details
conv_layers_info = extract_conv_layers_details(subgraph)

for layer_info in conv_layers_info:
    print(f"Layer Name: {layer_info['name']}")
    print(f"Layer Index: {layer_info['index']}")
    try:
        print(f"Weights Shape: {layer_info['weights_shape']}")
        print(f"Weights Dtype: {layer_info['weights_dtype']}")
        print(f"Padding: {layer_info['padding']}")
        print(f"Stride Height: {layer_info['stride_h']}")
        print(f"Stride Width: {layer_info['stride_w']}")
    except:
        pass
    print(f"Input Tensor Name: {layer_info['input_tensor_name']}")
    try:
        print(f"Weight Tensor Name: {layer_info['weight_tensor_name']}")
        print(f"Bias Tensor Name: {layer_info['bias_tensor_name']}")
    except:
        pass
    try:
        print(f"Padding: {layer_info['padding']}")
    except:
        pass
    print(f"Output Tensor Name: {layer_info['output_tensor_name']}")
    print("\n")


Layer Name: QUANTIZE_0
Layer Index: 0
Input Tensor Name: x.1
Output Tensor Name: input.26_te_transform


Layer Name: PAD_1
Layer Index: 1
Input Tensor Name: ['input.26_te_transform', 'input.26_te_transform_1_te_attr_2']
Output Tensor Name: input.26_te_transform_1_te_transform_2


Layer Name: CONV_2D_2
Layer Index: 2
Weights Shape: [16  3  3  3]
Weights Dtype: 0
Padding: 1
Stride Height: 2
Stride Width: 2
Input Tensor Name: input.26_te_transform_1_te_transform_2
Weight Tensor Name: fuse_attr_78
Bias Tensor Name: input0.10_te_attr_1
Padding: 1
Output Tensor Name: input.24_te_transform


Layer Name: PAD_3
Layer Index: 3
Input Tensor Name: ['input.24_te_transform', 'input.24_te_transform_1_te_attr_2']
Output Tensor Name: input.24_te_transform_1_te_transform_2


Layer Name: CONV_2D_4
Layer Index: 4
Weights Shape: [32  3  3 16]
Weights Dtype: 0
Padding: 1
Stride Height: 2
Stride Width: 2
Input Tensor Name: input.24_te_transform_1_te_transform_2
Weight Tensor Name: fuse_attr_79
Bias Tensor Na

In [ ]:
#save to csv
import csv
with open('conv_layers_info.csv', 'w', newline='') as csvfile:
    fieldnames = ['name', 'index', 'weights_shape', 'weights_dtype', 'padding', 'stride_h', 'stride_w', 'kernel_h', 'kernel_w', 'shape', 'input_tensor_name', 'weight_tensor_name', 'bias_tensor_name', 'output_tensor_name']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for layer_info in conv_layers_info:
        writer.writerow(layer_info)

# tensor details

In [ ]:
all_tensor_name = interpreter.get_tensor_details()

In [ ]:
all_tensor_value = interpreter.get_tensor(77)

In [ ]:
all_info = []
for i in range(len(all_tensor_name)):
   tensor = {}
   tensor['name'] = all_tensor_name[i]['name']
   tensor['index'] = i
   try:
      try:
          tensor['value'] = interpreter.get_tensor(i).transpose(1, 0, 2, 3).flatten().tolist()
      except:
          tensor['value'] = interpreter.get_tensor(i).flatten().tolist()
   except:

       pass

   all_info.append(tensor)

In [ ]:
#to_csv

# import csv
# with open('all_info_.csv', 'w', newline='') as csvfile:
#     fieldnames = ['name', 'index', 'value']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for layer_info in all_info:
#         writer.writerow(layer_info)
#conver to jsson
import json
with open('all_info_.json', 'w') as f:
    json.dump(all_info, f)